In [63]:
import os
import pandas as pd
from Data.database import BancodeDados,BancodeDados_cadastro
from modelo.dizimista import dizimista
from modelo.rua import Rua
import numpy as np

In [64]:
def obterNome(nomeArquivo:str):
    ponto = nomeArquivo.index(".")
    return nomeArquivo[:ponto]

In [65]:
cadastro = BancodeDados_cadastro()
cadastro.criar()
cadastro.inserirCadastro("Suely","Carolina98771139","Suely Aparecida","Comunidade Santa Mônica")

False

In [66]:
comunidade = BancodeDados("Comunidade Santa Mônica")
comunidade.criar()

In [67]:
documentos = os.listdir("DadosAno")

In [68]:
datas = {}
path = "DadosAno/"
for doc in documentos:
    ano = obterNome(doc)
    datas[ano] = pd.read_excel(path+doc)

In [69]:
anos = ["2021","2022","2023"]
meses = ["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]

In [70]:
def getNiver(string:str):
    if string == "12/10/1800":
        return "/"
    return string[:2] + "/" + string[3:5]

In [71]:
def inserirDataFrame(df:pd.DataFrame,ano:str):
    tam = len(df)
    for i in range(tam):
        linha = df.loc[i]
        comunidade.inserirDizimista(linha["Dizimista"],linha["N° Casa"],getNiver(linha["Aniversário"]),linha["Rua"])
        comunidade.inserirRua(linha['Rua'],linha['Zelador'])
        for mes in meses:
            if linha[mes]:
                comunidade.marcarContribuinte(linha["Dizimista"],linha["Rua"],linha["N° Casa"],mes,ano)

In [72]:
def nullTovazio(value)->str:
    if str(value) == "nan":
        return ""
    return str(value)

In [101]:
for ano in anos:
    datas[ano]["N° Casa"] = datas[ano]["N° Casa"].apply(lambda x: nullTovazio(x))
    datas[ano]['Aniversário'] = datas[ano]['Aniversário'].astype(str)
    dataFrame = datas[ano]
    inserirDataFrame(dataFrame,ano)